# Spatial Beam Patterns

In this task, you must plot the spatial beam pattern given the channel measurements on the antenna elements. Let the coordinate of the antenna elements i be defined by (xi , yi ), and the
corresponding channel measurement at antenna element i be h_i. 

Therefore, the spatial gain (power) at some point (x, y) on the 2D plane can be given by:

<img src="media/image4.png" alt="drawing" width="600"/> 


where N is the number of antenna elements, and λ is the wavelength of the RF signal.

However, due to the reflective nature of RFID transmissions, the above formula will need to be modified. 

You should write down the modified formula in your report, and explain it. Further, you should complete the code based on the above modified formula, in the function
`spatial_beam_pattern` to return the Power value at some point (x, y) in 2D space. 

This function should take the following inputs:

- The coordinate (x,y).
- The matrix ant enna_pos, where the i t h row represents the tuple (xi , yi ).
- The channel vector h, where hi represents the channel observed at the i t h antenna element.
- λ which is the wavelength of the RF signal

In [9]:
import numpy as np
import matplotlib.pyplot as plt
import os


def spatial_beam_pattern(x, y, antenna_pos, h, wavelength):

    # Extract xi and yi from antenna_pos
    xi = antenna_pos[:, 0]
    yi = antenna_pos[:, 1]

    # Compute distances from the point (x, y) to each antenna element
    distances = np.sqrt((x - xi)**2 + (y - yi)**2)

    # Compute the phase shifts due to the round-trip distances
    phases = (4 * np.pi / wavelength) * distances

    # Compute the steering vector
    aux = np.exp(1j * phases)
    channel = np.exp(-1j * h)

    # Compute the weighted sum
    summation = np.sum(aux *channel)

    # Compute the power as the squared magnitude of the summation
    power = np.abs(summation)**2

    return power


def get_source_channel(antenna_coord, source, wavelength):
    
    xi = antenna_coord[:, 0]
    yi = antenna_coord[:, 1]
    x0, y0 = source

    # Compute distances from the source to each antenna element
    distances = np.sqrt((x0 - xi)**2 + (y0 - yi)**2)

    # Compute the phase shifts due to the distances
    channels = (4 * np.pi / wavelength) * distances  # Factor of 4π for round-trip path

    return channels


def make_ant_pos(antenna_spacing, num_ant):
    """
    Generates antenna positions for a linear array.

    Parameters:
    - antenna_spacing : Spacing between antennas (meters)
    - num_ant         : Number of antenna elements

    Returns:
    - antenna_coord : N x 2 array of antenna positions (xi, yi)
    """
    xi = np.arange(num_ant) * antenna_spacing
    xi= xi - np.mean(xi)
    yi = np.zeros(num_ant)

    antenna_coord = np.column_stack((xi, yi))
    return antenna_coord


#### Testing on simulated data 
After completing the function, run the code cell below to test it on simulated data. The outputs should be saved to the `Results` folder and submitted along with your code

In [10]:
# Parameters
lambda_ = 4
antenna_spacing = [lambda_/4, lambda_/2, lambda_]
num_ant = 4

source = [1000000, 1000000]
x_grid = np.arange(-1000, 1001)
y_grid = np.flip(np.arange(0, 2001))

# Loop through antenna spacings
for l in range(3):
    antenna_coord = make_ant_pos(antenna_spacing[l], num_ant)
    channels = get_source_channel(antenna_coord, source, lambda_)

    matrix = np.zeros((len(x_grid), len(y_grid)))

    for j in range(len(x_grid)):
        for k in range(len(y_grid)):
            matrix[j, k] = spatial_beam_pattern(x_grid[j], y_grid[k], antenna_coord, channels, lambda_)

    matrix = np.transpose(matrix)

    plt.figure(dpi = 300, figsize=(10, 10))
    plt.imshow(matrix, extent=[x_grid.min(), x_grid.max(), y_grid.min(), y_grid.max()], cmap='gray', aspect='auto')
    plt.colorbar()
    plt.ylabel('Y Axis', fontsize=16)
    plt.xlabel('X Axis', fontsize=16)
    plt.title(f'Test Case Spatial Beam Pattern with Antenna Spacing {antenna_spacing[l]}')
    plt.savefig(f'Results/Result_test_spatial_beam{antenna_spacing[l]}.png', format='png')
    plt.close()

#### Testing on Data from RFID Hardware

Below you will find the paramters of the channel values and positions of three antenna elements. 
- Plot the spatial beam pattern considering only antenna 1 and 2
- Plot the spatial beam pattern considering only antenna 1 and 3

In [12]:
# Values of h1, h2 and h3 provided here
h1 = 66.2000  # In degrees
h2 = 109.6875  # In degrees
h3 = 164.4250  # In degrees

# Convert to radians
h1 = np.pi * h1 / 180
h2 = np.pi * h2 / 180
h3 = np.pi * h3 / 180

# Coordinates of Antenna Elements in 2D plane provided here
# Assume ant 1 is at origin and all other antennas lie on x axis
d = 7.62e-2
ant1 = np.array([0, 0])
ant2 = np.array([-d, 0])
ant3 = np.array([-2 * d, 0])


# Frequency and Lambda provided here
freq = 9.0275e8
wavelength = 3e8 / freq


In [11]:
import numpy as np
import matplotlib.pyplot as plt
import os

# Define x and y grids
x_grid = np.arange(-1000, 1001)  # From -1000 to 1000
y_grid = np.arange(0, 2001)      # From 0 to 2000

# Values of h1, h2, and h3 provided here (in degrees)
h1_deg = 66.2000
h2_deg = 109.6875
h3_deg = 164.4250

# Convert to radians
h1 = np.deg2rad(h1_deg)
h2 = np.deg2rad(h2_deg)
h3 = np.deg2rad(h3_deg)


# Antenna positions
d = 7.62e-2  # Antenna spacing
ant1 = np.array([0, 0])
ant2 = np.array([-d, 0])
ant3 = np.array([-2 * d, 0])

# Frequency and Wavelength
freq = 9.0275e8  # Frequency in Hz
wavelength = 3e8 / freq  # Wavelength in meters

# Define antenna_spacing and l for plot titles and filenames
antenna_spacing = [d]
l = 0  # Index in antenna_spacing list


# List of cases to process
cases = [
    {'antennas': [ant1, ant2], 'channels': [h1, h2], 'label': 'Antenna 1 and Antenna 2'},
    {'antennas': [ant1, ant3], 'channels': [h1, h3], 'label': 'Antenna 1 and Antenna 3'}
]

# Loop over the cases
for case in cases:
    antenna_coord = np.array(case['antennas'])
    channels = np.array(case['channels'])
    label = case['label']

    # Initialize the matrix for storing power values
    matrix = np.zeros((len(y_grid), len(x_grid)))

    # Compute the power at each grid point
    for j in range(len(x_grid)):
        for k in range(len(y_grid)):
            x = x_grid[j]
            y = y_grid[k]
            power = spatial_beam_pattern(x, y, antenna_coord, channels, wavelength)
            matrix[k, j] = power  # Note the order of indices for correct orientation

    # Convert power to dB scale for better visualization
    matrix_dB = 10 * np.log10(matrix / np.max(matrix))

    # Plotting
    plt.figure(figsize=(10, 8))
    plt.imshow(matrix_dB, extent=[x_grid.min(), x_grid.max(), y_grid.min(), y_grid.max()],
               cmap='jet', aspect='auto', origin='lower')
    plt.colorbar(label='Power (dB)')
    plt.ylabel('Y Axis (m)', fontsize=16)
    plt.xlabel('X Axis (m)', fontsize=16)
    plt.title(f'{label} Spatial Beam Pattern with Antenna Spacing {antenna_spacing[l]:.4f} m', fontsize=20)

    # Save the figure
    filename = f'Results/Result_test_spatial_beam_{label.replace(" ", "_")}_{antenna_spacing[l]:.4f}m.png'
    plt.savefig(filename, format='png')
    plt.close()


